In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization

es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

EPOCHS = 1000

2024-01-15 11:52:36.382257: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 11:52:36.426303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 11:52:37.012633: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('IMDB Dataset.csv', sep=",")

In [3]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
df['review'][49996]

'Bad plot, bad dialogue, bad acting, idiotic directing, the annoying porn groove soundtrack that ran continually over the overacted script, and a crappy copy of the VHS cannot be redeemed by consuming liquor. Trust me, because I stuck this turkey out to the end. It was so pathetically bad all over that I had to figure it was a fourth-rate spoof of Springtime for Hitler.<br /><br />The girl who played Janis Joplin was the only faint spark of interest, and that was only because she could sing better than the original.<br /><br />If you want to watch something similar but a thousand times better, then watch Beyond The Valley of The Dolls.'

In [5]:
import re
import string
import nltk
#nltk - natural language tool kit - bibliotek a do przetwarzania języka
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from tqdm import tqdm
from nltk import sent_tokenize,word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/jupyter-
[nltk_data]     tykfik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     tykfik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter-
[nltk_data]     tykfik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jupyter-
[nltk_data]     tykfik/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
main = []

# Storing all punctuations using RE library like !;,"% etc
re_puncs = re.compile('[%s]' % re.escape(string.punctuation))
# Storing all stop words like a, an, the, when, there, this etc
stop_word  = set(stopwords.words('english'))
stop_word.add("im")
stop_word.add("br")
print(stop_word)
print(re_puncs)


{'once', 'don', 'll', 'most', 'under', 'here', 'had', 'been', 'all', 'you', 'him', 'yourselves', 'whom', 'how', 'isn', 'than', 'into', "should've", 'that', 'during', 'aren', "didn't", 'very', 'from', 'too', "you'd", 'to', 'me', 'its', 'do', "couldn't", 'mustn', 'myself', 'having', 'it', 'his', "don't", 'now', "mightn't", 'out', 'hasn', 'couldn', 'yours', 'ain', 'against', 'our', "doesn't", 'those', 'being', 'after', "shan't", 'while', 'did', 'again', 'more', 'these', 'the', 'am', "hasn't", "you're", 'shan', 'until', 'there', 'my', 'd', 'he', 'off', 'themselves', 'were', 'what', "wasn't", 'such', 'each', 'yourself', 'i', 'y', 'them', "she's", 'be', 'because', "mustn't", 'who', 'nor', 've', 'im', 'over', 'with', 'through', 'before', 'weren', 'hers', 'of', "it's", 'can', 'has', 'no', 'just', 'won', 'wouldn', 'few', 'further', "you'll", 'for', 'was', 'should', 'she', 'wasn', "isn't", 'both', 'they', 'mightn', 'o', 'and', 'br', 'herself', "needn't", 'needn', 'shouldn', 'by', 'didn', 'then',

In [7]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable


In [8]:
sentences_listed = [line.split(" ") for line in df['review']]

import gensim

word_model = gensim.models.Word2Vec(sentences = sentences_listed, vector_size = 10)
words = list(word_model.wv.key_to_index)
print(len(words))

76833


In [9]:
word_model.wv.most_similar("marriage")

#sprawdzamy teraz połączenia semantyczne ze słowem "actress" wg tych recenzji
#on ogarnia sobie najczęstsze słowa, z którym to nasze występuje; są najbardziej zbliżone
#musimy to zrobić, żeby sprawdzić czy połączenia semantyczne są ok (w zależności od filmu/książki i ich kontekstu te
#połączenia mogą się trochę różnić)

[('victim', 0.9730309247970581),
 ('business', 0.9615693092346191),
 ('hire.', 0.9597921371459961),
 ('death,', 0.9595767259597778),
 ('heroine', 0.958457350730896),
 ('murder', 0.9545379877090454),
 ('body', 0.9530279636383057),
 ('demon', 0.9517441987991333),
 ('mask', 0.9503416419029236),
 ('position', 0.9487322568893433)]

lematyzacja - sprowadzanie słowa do jego podstawowej postaci; czyli w przypadku czasownika to będzie bezokolicznik

In [10]:
# Making Lemmatizing object
lem = WordNetLemmatizer()
# Using Porter Stemmer
p_stem = PorterStemmer()

# Traversing whole dataset
for i in tqdm(range(len(df['review']))):
    # Tokenization
    tokens = word_tokenize(str(df['review'][i]))
    # Converting all characters to lower case
    tokens = [w.lower() for w in tokens]
    # Remove all punctuations from sentenses
    tokens = [re_puncs.sub('', w) for w in tokens]
    # Checking all words is alphabets or not
    tokens = [i for i in tokens if i.isalpha()]
    # Removing all stop words from the sentenses
    tokens = [w for w in tokens if w not in stop_word]
    # Doing Lemmatizing of words
    tokens = [lem.lemmatize(w) for w in tokens]
    # Stemming process
    tokens = [p_stem.stem(w) for w in tokens]
    # Finally convert to string
    r = ' '.join(tokens)
    # Storing the final string into main list
    main.append(r)
    
#https://www.kaggle.com/code/tanujdhiman/twitter-sentiment-analysis

100%|██████████| 50000/50000 [02:52<00:00, 290.18it/s]


In [11]:
df['translated'] = main

In [12]:
df

,review,sentiment,translated
0,One of the other reviewers has mentioned that ...,positive,one review mention watch oz episod hook right ...
1,A wonderful little production. <br /><br />The...,positive,wonder littl product film techniqu unassum old...
2,I thought this was a wonderful way to spend ti...,positive,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,negative,basic famili littl boy jake think zombi closet...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visual stun film...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movi right good job nt creativ origin ...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogu bad act idiot direct anno...
49997,I am a Catholic taught in parochial elementary...,negative,cathol taught parochi elementari school nun ta...
49998,I'm going to have to disagree with the previou...,negative,go disagre previou comment side maltin one sec...


In [13]:
df['translated'][0]

'one review mention watch oz episod hook right exactli happen first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use word call oz nicknam given oswald maximum secur state penitentari focu mainli emerald citi experiment section prison cell glass front face inward privaci high agenda em citi home mani aryan muslim gangsta latino christian italian irish scuffl death stare dodgi deal shadi agreement never far away would say main appeal show due fact go show would nt dare forget pretti pictur paint mainstream audienc forget charm forget romanc oz nt mess around first episod ever saw struck nasti surreal could nt say readi watch develop tast oz got accustom high level graphic violenc violenc injustic crook guard sold nickel inmat kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch oz may becom comfort uncomfort view that get touch d

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['translated'])

In [15]:
print(len(tokenizer.word_counts))

101286


In [16]:
counter = 0
for key, value in tokenizer.word_counts.items():
    if value > 40:
        print(key, value)
        counter+=1
        
print(counter)
#wszystkie słowa, które pojawiły się więcej niż 40 razy

one 54100
review 4292
mention 3013
watch 27683
oz 268
episod 4980
hook 587
right 6890
exactli 1964
happen 7166
first 17257
thing 16459
struck 267
brutal 937
scene 21204
violenc 2063
set 7645
word 3629
go 22621
trust 731
show 19740
faint 117
heart 2709
timid 51
pull 1849
punch 536
regard 957
drug 1673
sex 3297
hardcor 255
classic 4255
use 9079
call 5461
nicknam 83
given 3568
maximum 103
secur 494
state 2070
focu 1324
mainli 774
citi 2466
experiment 209
section 459
prison 1407
cell 461
glass 503
front 1232
face 4264
high 3875
agenda 186
em 243
home 3767
mani 13457
muslim 202
gangsta 41
latino 91
christian 945
italian 1144
irish 384
death 4229
stare 421
dodgi 91
deal 2595
shadi 72
agreement 79
never 12907
far 5802
away 5458
would 26492
say 14969
main 4637
appeal 1367
due 1792
fact 7309
nt 66249
dare 648
forget 1653
pretti 7226
pictur 3790
paint 988
mainstream 378
audienc 5207
charm 1766
romanc 1452
mess 1520
around 7090
ever 11818
saw 6344
nasti 705
surreal 525
could 18543
readi 671
devel

In [17]:
tokenizer.texts_to_sequences(df['translated'])

#dataset zamieniony na liczby

[[4,
  225,
  346,
  12,
  2773,
  188,
  1573,
  113,
  543,
  103,
  31,
  35,
  2784,
  2773,
  1059,
  10984,
  18,
  505,
  90,
  113,
  289,
  17,
  1310,
  20,
  4908,
  389,
  8224,
  20,
  565,
  1674,
  1039,
  630,
  316,
  505,
  2895,
  228,
  72,
  289,
  166,
  2773,
  6087,
  293,
  10782,
  5314,
  1784,
  502,
  37035,
  798,
  1252,
  16415,
  427,
  3323,
  1886,
  745,
  1883,
  1758,
  852,
  227,
  17006,
  12039,
  259,
  3587,
  2992,
  427,
  274,
  42,
  10077,
  3395,
  9309,
  5756,
  1051,
  897,
  2150,
  14532,
  230,
  2004,
  5757,
  403,
  6638,
  6278,
  50,
  152,
  167,
  13,
  38,
  201,
  772,
  20,
  588,
  99,
  17,
  20,
  13,
  3,
  1451,
  644,
  101,
  271,
  1018,
  2179,
  179,
  644,
  597,
  644,
  730,
  2773,
  3,
  696,
  107,
  31,
  188,
  55,
  128,
  2784,
  1357,
  1705,
  24,
  3,
  38,
  1408,
  12,
  358,
  898,
  2773,
  100,
  7544,
  259,
  458,
  1161,
  505,
  505,
  5026,
  2855,
  1695,
  2557,
  18363,
  4259,
  98,
 

In [18]:
tokenized_reviews = tokenizer.texts_to_sequences(df['translated'])
print(tokenized_reviews[0])

[4, 225, 346, 12, 2773, 188, 1573, 113, 543, 103, 31, 35, 2784, 2773, 1059, 10984, 18, 505, 90, 113, 289, 17, 1310, 20, 4908, 389, 8224, 20, 565, 1674, 1039, 630, 316, 505, 2895, 228, 72, 289, 166, 2773, 6087, 293, 10782, 5314, 1784, 502, 37035, 798, 1252, 16415, 427, 3323, 1886, 745, 1883, 1758, 852, 227, 17006, 12039, 259, 3587, 2992, 427, 274, 42, 10077, 3395, 9309, 5756, 1051, 897, 2150, 14532, 230, 2004, 5757, 403, 6638, 6278, 50, 152, 167, 13, 38, 201, 772, 20, 588, 99, 17, 20, 13, 3, 1451, 644, 101, 271, 1018, 2179, 179, 644, 597, 644, 730, 2773, 3, 696, 107, 31, 188, 55, 128, 2784, 1357, 1705, 24, 3, 38, 1408, 12, 358, 898, 2773, 100, 7544, 259, 458, 1161, 505, 505, 5026, 2855, 1695, 2557, 18363, 4259, 98, 457, 11, 167, 19, 969, 673, 628, 4259, 94, 745, 3985, 588, 288, 593, 961, 745, 336, 12, 2773, 119, 134, 1764, 2561, 223, 1405, 11, 432, 3334, 378]
